## How often does President Trump tweet each day?

In [3]:
import altair as alt
import pandas as pd
import matplotlib as mpl
import numpy as np
import json
import pytz
from datetime import datetime

### Get the latest 1,000 tweets from the thetrumparchive.com API endpoint

In [4]:
latest = pd.read_json('https://www.thetrumparchive.com/latest-tweets', dtype={'id':str})

In [5]:
latest.head()

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,1347569870578266115,"To all of those who have asked, I will not be ...",False,False,Twitter for iPhone,540569,71751,2021-01-08 15:44:28+00:00,False
1,1347555316863553542,"The 75,000,000 great American Patriots who vot...",False,False,Twitter for iPhone,448926,80223,2021-01-08 14:46:38+00:00,False
2,1347334804052844550,https://t.co/csX07ZVWGe,False,False,Twitter Media Studio,578503,120265,2021-01-08 00:10:24+00:00,False
3,1346954970910707712,These are the things and events that happen wh...,False,True,Twitter for iPhone,0,0,2021-01-06 23:01:04+00:00,False
4,1346928882595885058,https://t.co/Pm2PKV0Fp3,False,True,Twitter for iPhone,0,0,2021-01-06 21:17:24+00:00,False


In [7]:
latest[latest['text'].str.contains('ballots')]

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
9,1346845109736845318,These scoundrels are only toying with the @sen...,False,False,Twitter for iPhone,181861,32001,2021-01-06 15:44:31+00:00,False
12,1346818855298072576,"They just happened to find 50,000 ballots late...",False,False,Twitter for iPhone,284555,55789,2021-01-06 14:00:12+00:00,False
24,1346685023861272580,Just happened to have found another 4000 ballo...,False,False,Twitter for iPhone,223082,39799,2021-01-06 05:08:24+00:00,False
90,1345731043861659650,I spoke to Secretary of State Brad Raffensperg...,False,False,Twitter for iPhone,132702,28368,2021-01-03 13:57:37+00:00,False
117,1345404684723507200,....Just a small portion of these votes give U...,False,False,Twitter for iPhone,138142,33088,2021-01-02 16:20:47+00:00,False
175,1344160787384971264,"....that, quite frankly, didn’t have much of a...",False,False,Twitter for iPhone,128681,25801,2020-12-30 05:57:59+00:00,False
410,1338718436508241923,RT @KanekoaTheGreat: DOMINION - NEVADA🚨\n\nThe...,True,True,Twitter for iPhone,0,17116,2020-12-15 05:32:02+00:00,False
420,1338483200046354434,“Why did the Swing States stop counting in the...,False,False,Twitter for iPhone,269237,62408,2020-12-14 13:57:17+00:00,False
430,1338254787020787712,".....and many others voted illegally. Also, ma...",False,False,Twitter for iPhone,156467,31232,2020-12-13 22:49:39+00:00,False
438,1338246164756520961,...Tens of millions of haphazardly ballots sen...,False,False,Twitter for iPhone,124921,22337,2020-12-13 22:15:23+00:00,False


In [8]:
latest.sort_values('date', ascending=False).head()

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,1347569870578266115,"To all of those who have asked, I will not be ...",False,False,Twitter for iPhone,540569,71751,2021-01-08 15:44:28+00:00,False
1,1347555316863553542,"The 75,000,000 great American Patriots who vot...",False,False,Twitter for iPhone,448926,80223,2021-01-08 14:46:38+00:00,False
2,1347334804052844550,https://t.co/csX07ZVWGe,False,False,Twitter Media Studio,578503,120265,2021-01-08 00:10:24+00:00,False
3,1346954970910707712,These are the things and events that happen wh...,False,True,Twitter for iPhone,0,0,2021-01-06 23:01:04+00:00,False
4,1346928882595885058,https://t.co/Pm2PKV0Fp3,False,True,Twitter for iPhone,0,0,2021-01-06 21:17:24+00:00,False


In [9]:
#get latest
trump_past = pd.read_json('input/tweets_11-06-2020.json')

In [10]:
trump_extra = pd.read_json('input/tweets_11-04-2020_01-08-2021.json')

### Combine the latest 1,000 with past archive

In [11]:
src = pd.concat([latest, trump_past, trump_extra])

In [12]:
src = src.drop_duplicates(subset='text')

### Clean up dates

In [13]:
src.head()

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,1347569870578266115,"To all of those who have asked, I will not be ...",False,False,Twitter for iPhone,540569,71751,2021-01-08 15:44:28+00:00,False
1,1347555316863553542,"The 75,000,000 great American Patriots who vot...",False,False,Twitter for iPhone,448926,80223,2021-01-08 14:46:38+00:00,False
2,1347334804052844550,https://t.co/csX07ZVWGe,False,False,Twitter Media Studio,578503,120265,2021-01-08 00:10:24+00:00,False
3,1346954970910707712,These are the things and events that happen wh...,False,True,Twitter for iPhone,0,0,2021-01-06 23:01:04+00:00,False
4,1346928882595885058,https://t.co/Pm2PKV0Fp3,False,True,Twitter for iPhone,0,0,2021-01-06 21:17:24+00:00,False


In [14]:
src['eastern_created_at'] = pd.to_datetime(src['date'], utc=True)

In [15]:
src['eastern_created_at'] = pd.to_datetime(src['date'], utc=True).dt.tz_convert('America/Los_Angeles')

In [16]:
src['date'] = pd.to_datetime(src['eastern_created_at']).dt.strftime('%m/%d/%Y')
src['date'] = pd.to_datetime(src['date'])
src['year'] = src['eastern_created_at'].dt.year
src['month'] = src['eastern_created_at'].dt.month
src['day'] = src['eastern_created_at'].dt.day
src['hour'] = src['eastern_created_at'].dt.hour
src['minute'] = src['eastern_created_at'].dt.minute
src['time'] = src['eastern_created_at'].dt.time
src['id'] = src['id'].astype(str)
src['year'] = src['year'].astype(str)
src['month'] = src['month'].astype(str)
src['day'] = src['day'].astype(str)
src['hour'] = src['hour'].astype(str)

In [17]:
src.dtypes

id                                                 object
text                                               object
isRetweet                                          object
isDeleted                                          object
device                                             object
favorites                                           int64
retweets                                            int64
date                                       datetime64[ns]
isFlagged                                          object
eastern_created_at    datetime64[ns, America/Los_Angeles]
year                                               object
month                                              object
day                                                object
hour                                               object
minute                                              int64
time                                               object
dtype: object

In [18]:
november2020 = src[(src['year'] == '2020') & (src['month'] == '11')]

In [19]:
november2020.groupby(['date']).size()

date
2020-11-01    60
2020-11-02    85
2020-11-03    18
2020-11-04    14
2020-11-05    17
2020-11-06    27
2020-11-07    12
2020-11-08    19
2020-11-09    32
2020-11-10    37
2020-11-11    24
2020-11-12    48
2020-11-13    41
2020-11-14    38
2020-11-15    45
2020-11-16    41
2020-11-17    22
2020-11-18    26
2020-11-19    22
2020-11-20    24
2020-11-21    38
2020-11-22    17
2020-11-23     4
2020-11-24    41
2020-11-25    43
2020-11-26    14
2020-11-27    33
2020-11-28     8
2020-11-29    16
2020-11-30    29
dtype: int64

In [20]:
src.drop(['eastern_created_at'], axis=1, inplace=True)

In [22]:
src.reset_index().to_csv('/Users/mstiles/github/notebooks/\
trump-tweets/output/realdonaldtrump.csv', index=False)